In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from commons.azure_authentication import AzureAuthHelper
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent
from azure.ai.agents.models import AgentsNamedToolChoice, AgentsNamedToolChoiceType, FunctionName
from dotenv import load_dotenv

In [3]:
load_dotenv(override=True)

True

In [4]:
AZURE_AI_FOUNDRY_PROJECT_ENDPOINT = os.getenv("AZURE_AI_FOUNDRY_PROJECT_ENDPOINT")
AGENT_NAME = os.getenv("AGENT_NAME", "my-agentic-rag-agent")
AGENT_MODEL = os.getenv("AGENT_MODEL", "gpt-5-nano")

In [5]:
project_client = AIProjectClient(
    endpoint=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
    credential= AzureAuthHelper(method="default").get_credential()
)

2025-11-12 16:44:11 [INFO] Using DefaultAzureCredential


In [ ]:
instructions = """
A Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].
If you do not have the answer, respond with "I don't know".
"""
agent = project_client.agents.create_agent(
    model=AGENT_MODEL,
    name=AGENT_NAME,
    instructions=instructions
)

print(f"AI agent '{AGENT_NAME}' created or updated successfully")

In [6]:
agent_client = KnowledgeAgentRetrievalClient(
    endpoint=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
    agent_name="Agent12112025", 
    credential=AzureAuthHelper(method="default").get_credential()
    )

2025-11-12 16:44:25 [INFO] Using DefaultAzureCredential


In [ ]:
thread = project_client.agents.threads.create()
retrieval_results = {}

## **AGENT FUNCTIONS**

In [ ]:
def agentic_retrieval() -> str:
    """
        Searches a NASA e-book about images of Earth at night and other science related facts.
        The returned string is in a JSON format that contains the reference id.
        Be sure to use the same format in your agent's response
        You must refer to references by id number
    """
    # Take the last 5 messages in the conversation
    messages = project_client.agents.messages.list(thread.id, limit=5, order=ListSortOrder.DESCENDING)
    # Reverse the order so the most recent message is last
    messages = list(messages)
    messages.reverse()
    retrieval_result = agent_client.retrieve(
        retrieval_request=KnowledgeAgentRetrievalRequest(
            messages=[
                    KnowledgeAgentMessage(
                        role=m["role"],
                        content=[KnowledgeAgentMessageTextContent(text=m["content"])]
                    ) for m in messages if m["role"] != "system"
            ]
        )
    )

    # Associate the retrieval results with the last message in the conversation
    last_message = messages[-1]
    retrieval_results[last_message.id] = retrieval_result

    # Return the grounding response to the agent
    return retrieval_result.response[0].content[0].text

In [ ]:
functions = FunctionTool({agentic_retrieval})
toolset = ToolSet()
toolset.add(functions)
project_client.agents.enable_auto_function_calls(toolset)

## **START THE CONVERSATION**

In [ ]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
        Why do suburban belts display larger December brightening than 
        urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, 
        whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
)

In [ ]:
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_choice=AgentsNamedToolChoice(type=AgentsNamedToolChoiceType.FUNCTION, function=FunctionName(name="agentic_retrieval")),
    toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role="assistant").text.value

print("Agent response:", output.replace(".", "\n"))